In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# 事件
train_event = pd.read_csv('../jet_complex_data/complex_train_R04_event.csv')
test_event = pd.read_csv('../jet_complex_data/complex_test_R04_event.csv')
# 喷注
train_jet = pd.read_csv('../jet_complex_data/complex_train_R04_jet.csv')
test_jet = pd.read_csv('../jet_complex_data/complex_test_R04_jet.csv')
# 粒子
train_particle = pd.read_csv('../jet_complex_data/complex_train_R04_particle.csv')
test_particle = pd.read_csv('../jet_complex_data/complex_test_R04_particle.csv')

In [ ]:
train_event.head(3)

In [ ]:
train_particle.head(3)

In [ ]:
print('--- Data Size')
print('event: train %d, test %d' % (len(train_event), len(test_event)))
print('jet: train %d, test %d' % (len(train_jet), len(test_jet)))
print('particle: train %d, test %d' % (len(train_particle), len(test_particle)))
print('--- Amount')
print('event: train %d, test %d' % (train_event.event_id.nunique(), test_event.event_id.nunique()))
print('jet: train %d, test %d' % (train_jet.jet_id.nunique(), test_jet.jet_id.nunique()))
print('event in jet: train %d, test %d' % (train_jet.event_id.nunique(), test_jet.event_id.nunique()))
print('jet in particle: train %d, test %d' % (train_particle.jet_id.nunique(), test_particle.jet_id.nunique()))
print('--- NaN')
print('event: train %d, test %d' % (train_event.isnull().sum().sum(), test_event.isnull().sum().sum()))
print('jet: train %d, test %d' % (train_jet.isnull().sum().sum(), test_jet.isnull().sum().sum()))
print('particle: train %d, test %d' % (train_particle.isnull().sum().sum(), test_particle.isnull().sum().sum()))

In [ ]:
ax = train_jet.label.value_counts(normalize=True).plot(kind='bar', title='Distribution of jet label')

In [ ]:
event_label = train_jet.groupby('event_id')['label'].agg('nunique')
print('Max number of jet types in a event: ', event_label.max())

In [ ]:
ax = train_jet.groupby('event_id')['label'].nth(0).value_counts(normalize=True).plot(kind='bar',                                                                           title='Distribution of event label')

In [ ]:
plt.figure(figsize=(12, 4))
ax = plt.subplot(1,2,1)
ax.set_title('boxplot: jet direction')
sns.boxplot(data=train_jet[['jet_px', 'jet_py', 'jet_pz']])
plt.subplot(1,2,2)
ax = sns.boxplot(y='jet_px',x='label',data=train_jet)
ax.set_title('boxplot: jet_x of different jet type')
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
ax = plt.subplot(1,2,1)
ax.set_title('boxplot: jet energy & mass')
sns.boxplot(data=train_jet[['jet_energy', 'jet_mass']])
plt.subplot(1,2,2)
ax = sns.boxplot(y='jet_energy',x='label',data=train_jet)
ax.set_title('boxplot: jet_energy of different jet type')
plt.show()

In [ ]:
ax = train_particle.particle_category.value_counts(normalize=True).plot(kind='bar', title='Distribution of particle type')

In [ ]:
print('粒子质量统计值')
train_particle.groupby('particle_category')['particle_mass'].agg(['min', 'max', 'mean'])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import pickle
import time

In [ ]:
import catboost as cbt

In [ ]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_jet = pd.read_csv('../jet_complex_data/complex_train_R04_jet.csv')
test_jet = pd.read_csv('../jet_complex_data/complex_test_R04_jet.csv')

In [ ]:
train_event = pd.read_csv('../jet_complex_data/complex_train_R04_event.csv')
test_event = pd.read_csv('../jet_complex_data/complex_test_R04_event.csv')

In [ ]:
train_particle = pd.read_csv('../jet_complex_data/complex_train_R04_particle.csv')
test_particle = pd.read_csv('../jet_complex_data/complex_test_R04_particle.csv')

In [ ]:
# 构造事件标签
def gen_event_label(event, jet):
    assert jet.groupby('event_id')['label'].nunique().max() == 1
    event_label = jet.groupby('event_id')['label'].first().reset_index()
    event_label = event[['event_id']].merge(event_label, 'left', 'event_id')
    return event_label

In [ ]:
# 特征：1）对事件包含的喷注的物理属性求统计值。
def gen_jet_feat(event, jet):
    
    feat = jet[['event_id', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_mass', 'number_of_particles_in_this_jet']]
    feat['jet_p'] = (feat['jet_px'] ** 2 + feat['jet_py'] ** 2 + feat['jet_pz'] ** 2) ** 0.5
    feat['jet_cos(x)'] = feat['jet_px'] / feat['jet_p']
    feat['jet_cos(y)'] = feat['jet_py'] / feat['jet_p']
    feat['jet_cos(z)'] = feat['jet_pz'] / feat['jet_p']
    feat['jet_angle(x)'] = np.arccos(feat['jet_cos(x)'])
    feat['jet_angle(y)'] = np.arccos(feat['jet_cos(y)'])
    feat['jet_angle(z)'] = np.arccos(feat['jet_cos(z)'])
    feat['jet_energy/jet_mass'] = feat['jet_energy'] / feat['jet_mass']
    cols = ['jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_mass', 'number_of_particles_in_this_jet', 'jet_energy/jet_mass',
            'jet_p', 'jet_cos(x)', 'jet_cos(y)', 'jet_cos(z)', 'jet_angle(x)', 'jet_angle(y)', 'jet_angle(z)']
    
    st = ['min', 'max', 'mean', 'std', 'sum']
    st_cols = [(c + '_' + s) for c in cols for s in st]
    feat = feat.groupby('event_id')[cols].agg(st).reset_index()
    feat.columns = ['event_id'] + st_cols
    
    feat = event[['event_id', 'number_of_jet_in_this_event']].merge(feat, 'left', 'event_id')
    feat = feat.drop(columns=['event_id'])
    
    return feat

In [ ]:
# 特征：2）对事件包含的粒子的物理属性求统计值。
def gen_particle_feat(event, jet, particle):
    
    # 事件的粒子属性特征
    particle = particle.copy().merge(jet[['jet_id', 'event_id']], 'left', 'jet_id')
    particle['particle_p'] = (particle['particle_px'] ** 2 + particle['particle_py'] ** 2 + particle['particle_pz'] ** 2) ** 0.5
    particle['particle_cos(x)'] = particle['particle_px'] / particle['particle_p']
    particle['particle_cos(y)'] = particle['particle_py'] / particle['particle_p']
    particle['particle_cos(z)'] = particle['particle_pz'] / particle['particle_p']
    particle['particle_angle(x)'] = np.arccos(particle['particle_cos(x)'])
    particle['particle_angle(y)'] = np.arccos(particle['particle_cos(y)'])
    particle['particle_angle(z)'] = np.arccos(particle['particle_cos(z)'])
    particle['particle_energy/particle_mass'] = particle['particle_energy'] / particle['particle_mass']

    cols = ['particle_px', 'particle_py', 'particle_pz', 'particle_energy', 'particle_mass', 
            'particle_p', 'particle_cos(x)', 'particle_cos(y)', 'particle_cos(z)',
            'particle_angle(x)', 'particle_angle(y)', 'particle_angle(z)', 'particle_energy/particle_mass']

    st = ['min', 'max', 'mean', 'std', 'sum']
    st_cols = [(c + '_e_' + s) for c in cols for s in st]
    particle_st = particle.groupby('event_id')[cols].agg(st).reset_index()
    particle_st.columns = ['event_id'] + st_cols

    feat = event[['event_id']].merge(particle_st, 'left', 'event_id')
    
    # 事件的粒子类别特征
    particle['1'] = 1
    cat_cnt = particle.pivot_table(index='event_id', columns='particle_category', values='1', aggfunc='sum').reset_index()
    cat_cols = ['cat_e_%d' % i  for i in range(14)]
    cat_cnt.columns = ['event_id'] + ['cat_e_%d' % i  for i in range(14)]
    cat_cnt['cat_e_sum'] = cat_cnt[cat_cols].sum(axis=1)
    cat_cnt_rate = cat_cnt[cat_cols] / cat_cnt['cat_e_sum'].values.reshape((-1, 1))
    cat_cnt_rate.columns = ['%s_e_rate' % c for c in cat_cols]
    cat_cnt = pd.concat([cat_cnt, cat_cnt_rate], axis=1)

    feat = feat.merge(cat_cnt, 'left', 'event_id')

    feat = feat.drop(columns=['event_id'])
    
    return feat

In [ ]:
# 特征：3）对喷注包含的粒子的物理属性求统计值，再对事件求统计值。
def gen_jet_particle_feat(event, jet, particle):
    
    # 喷注的粒子属性特征
    particle = particle.copy() # merge(jet[['jet_id', 'event_id']], 'left', 'jet_id')
    particle['particle_p'] = (particle['particle_px'] ** 2 + particle['particle_py'] ** 2 + particle['particle_pz'] ** 2) ** 0.5
    particle['particle_cos(x)'] = particle['particle_px'] / particle['particle_p']
    particle['particle_cos(y)'] = particle['particle_py'] / particle['particle_p']
    particle['particle_cos(z)'] = particle['particle_pz'] / particle['particle_p']
    particle['particle_angle(x)'] = np.arccos(particle['particle_cos(x)'])
    particle['particle_angle(y)'] = np.arccos(particle['particle_cos(y)'])
    particle['particle_angle(z)'] = np.arccos(particle['particle_cos(z)'])
    particle['particle_energy/particle_mass'] = particle['particle_energy'] / particle['particle_mass']

    cols = ['particle_px', 'particle_py', 'particle_pz', 'particle_energy', 'particle_mass', 
            'particle_p', 'particle_cos(x)', 'particle_cos(y)', 'particle_cos(z)',
            'particle_angle(x)', 'particle_angle(y)', 'particle_angle(z)', 'particle_energy/particle_mass']

    st = ['min', 'max', 'mean', 'std', 'sum']
    st_cols = [(c + '_j_' + s) for c in cols for s in st]
    particle_st = particle.groupby('jet_id')[cols].agg(st).reset_index()
    particle_st.columns = ['jet_id'] + st_cols
    
    # 喷注的粒子类别特征
    particle['1'] = 1
    cat_cnt = particle.pivot_table(index='jet_id', columns='particle_category', values='1', aggfunc='sum').reset_index()
    cat_cols = ['cat_j_%d' % i  for i in range(14)]
    cat_cnt.columns = ['jet_id'] + ['cat_j_%d' % i  for i in range(14)]
    cat_cnt['cat_j_sum'] = cat_cnt[cat_cols].sum(axis=1)
    cat_cnt_rate = cat_cnt[cat_cols] / cat_cnt['cat_j_sum'].values.reshape((-1, 1))
    cat_cnt_rate.columns = ['%s_j_rate' % c for c in cat_cols]
    cat_cnt = pd.concat([cat_cnt, cat_cnt_rate], axis=1)

    feat = jet[['event_id', 'jet_id']].merge(particle_st, 'left', 'jet_id').merge(cat_cnt, 'left', 'jet_id')
    
    # 对事件求上述喷注特征的统计值
    cols = [c for c in feat.columns if not c in ['event_id', 'jet_id']]
    st = ['min', 'max', 'mean', 'std', 'sum']
    st_cols = [(c + '_e_' + s) for c in cols for s in st]
    feat = feat.groupby('event_id')[cols].agg(st).reset_index()
    feat.columns = ['event_id'] + st_cols
    
    feat = event[['event_id']].merge(feat, 'left', 'event_id')
    feat = feat.drop(columns=['event_id'])
    
    return feat

In [ ]:
# 事件标签
event_label = gen_event_label(train_event, train_jet)

In [ ]:
# 特征 1
train_jet_feat = gen_jet_feat(train_event, train_jet)
test_jet_feat = gen_jet_feat(test_event, test_jet)

In [ ]:
# 特征 2
train_particle_feat = gen_particle_feat(train_event, train_jet, train_particle)
test_particle_feat = gen_particle_feat(test_event, test_jet, test_particle)

In [ ]:
# 特征 3
train_jet_particle_feat = gen_jet_particle_feat(train_event, train_jet, train_particle)
test_jet_particle_feat = gen_jet_particle_feat(test_event, test_jet, test_particle)

In [ ]:
# 合并特征
train_data = pd.concat([train_jet_feat, train_particle_feat, train_jet_particle_feat], axis=1)
test_data = pd.concat([test_jet_feat, test_particle_feat, test_jet_particle_feat], axis=1)

In [ ]:
used_feats = list(train_data.columns)
x_train = train_data
x_test = test_data

print(len(used_feats))
print(used_feats)
print(x_train.shape)
print(x_test.shape)

In [ ]:
label = 'label'
lbl_enc = LabelEncoder()
event_label[label] = lbl_enc.fit_transform(event_label[label])
print(lbl_enc.classes_)
print(lbl_enc.transform(lbl_enc.classes_))
y_train = event_label[label]

In [ ]:
del train_jet_feat, train_particle_feat, train_jet_particle_feat
del test_jet_feat, test_particle_feat, test_jet_particle_feat

In [ ]:
lbr = LabelBinarizer().fit(y_train)
def auc_metric(y_true, y_pred):
    y_true = lbr.transform(y_true)
    y_pred = y_pred.reshape((4, -1)).T
    y_pred = lbr.transform(np.argmax(y_pred, axis=1))
    score = roc_auc_score(y_true=y_true, y_score=y_pred, average='macro')
    return 'auc', score, True

In [ ]:
tic = time.time()

test_pred = np.zeros((len(x_test), 4))
oof_pred = np.zeros((len(x_train), 4))

feat_imp = pd.DataFrame(used_feats, columns=['feat'])
feat_imp['imp'] = 0
scores = []

kfold = StratifiedKFold(n_splits=5, random_state=12306, shuffle=True)
for i, (trn_idx, val_idx) in enumerate(kfold.split(X=x_train, y=y_train)):

    print('-' * 88)
    print('Fold %d:' % i)
    
    x_trn, y_trn = x_train.iloc[trn_idx], y_train.iloc[trn_idx]
    x_val, y_val = x_train.iloc[val_idx], y_train.iloc[val_idx]
    
    trn_pool = cbt.Pool(x_trn, y_trn)
    val_pool = cbt.Pool(x_val, y_val)
    model = cbt.CatBoostClassifier(iterations=100000, learning_rate=0.1, eval_metric='MultiClass',# depth=10,
                               use_best_model=True, random_seed=2020, logging_level='Verbose', 
                               task_type='GPU', devices='0', early_stopping_rounds=200, loss_function='MultiClass', 
                               )
#     model.set_params(**params)
    model.fit(trn_pool, eval_set=val_pool, verbose=100)

    pickle.dump(file=open('./models/cbt_model_%d.pkl' % i, 'wb'), obj=model)
    feat_imp['imp'] += (model.feature_importances_ / 5)
    scores.append(model.best_score_['validation']['MultiClass'])
    test_pred += (model.predict_proba(x_test) / 5)
    oof_pred[val_idx] = model.predict_proba(x_val)
    
    del x_trn, y_trn, x_val, y_val
    del trn_pool, val_pool

toc = time.time()
print('times: %f' % (toc - tic))

In [ ]:
print('loss： %s' % scores)
print('mean loss: %f' % np.mean(scores))
print('acc: %f' % accuracy_score(y_train, np.argmax(oof_pred, axis=1)))
print('auc: %f' % roc_auc_score(y_true=lbr.transform(y_train),
                                y_score=lbr.transform(np.argmax(oof_pred, axis=1)),
                                average='macro'))

In [ ]:
# 以喷注为主体重新计算AUC
event_pred = train_event[['event_id']]
event_pred['pred'] = np.argmax(oof_pred, axis=1)
jet_pred = train_jet[['event_id']].merge(event_pred, 'left', 'event_id')['pred'].values
jet_label = lbl_enc.transform(train_jet['label'])
print('acc: %f' % accuracy_score(jet_label, jet_pred))
print('auc: %f' % roc_auc_score(y_true=lbr.transform(jet_label),
                                y_score=lbr.transform(jet_pred),
                                average='macro'))
# 0.766

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(5,10))
feat_imp= feat_imp.sort_values(by='imp', ascending=True)[-50:]
plt.barh(feat_imp['feat'], feat_imp['imp'])

In [ ]:
event_pred = test_event[['event_id']]
event_pred['pred'] = np.argmax(test_pred, axis=1)
jet_pred = test_jet[['event_id']].merge(event_pred, 'left', 'event_id')['pred'].values

In [ ]:
submission = pd.DataFrame()
submission['id'] = test_jet['jet_id']
submission['label'] = lbl_enc.inverse_transform(jet_pred)
submission.head()

In [ ]:
submission.to_csv('./result.csv', index=False)